In [2]:
# Crawl content, title, push and web for all of PTT board.
import time
import requests
import csv
from bs4 import BeautifulSoup
import os
import sys
import urllib3
# Disable the checking of certification.
urllib3.disable_warnings(False)

page_index = 2000 #開始頁面
end_index = 6463 + 1 #結束葉面
start_post = 0 #貼文起始index (-5公告)

In [3]:
err_max = 10
now_folder_len = 2000
now_folder_cnt = 0

start_index = page_index
#end_index = 3697
ptt_web = 'https://www.ptt.cc'
board = 'movie'
board_web_tail = '/bbs/' + board + '/index'
board_web = ptt_web + board_web_tail
user_name = 'user'
base_dir = 'C:/Users/'+user_name+'/Raw data/PTT/'
#base_dir = './drive/Raw data/PTT/'
over18_web = ptt_web + '/ask/over18'
payload18 = {
    'from':board_web_tail +'.html',
    'yes':'yes'
}

# Build folder for PTT content.
now_time = time.time()
print(now_time)
nowtime_dir = os.path.join(base_dir, board+'_'+str(now_time))
if not os.path.exists(nowtime_dir):
    os.makedirs(nowtime_dir)
content_dir = os.path.join(nowtime_dir, 'content')
if not os.path.exists(content_dir):
    os.makedirs(content_dir)
push_dir = os.path.join(nowtime_dir, 'push')
if not os.path.exists(push_dir):
    os.makedirs(push_dir)
csv_dir = nowtime_dir+'/'+str(now_time)+'.csv'

def row_1st_keep(input_text):
    row_1st_end = input_text.find('\n')
    if row_1st_end < 0:
        return input_text[0:-1]
    else:
        return input_text[0:row_1st_end]

po_ip = ''
f_list = open(csv_dir ,'a', encoding = 'utf-8-sig')
w_list = csv.writer(f_list)
postInfo = ['post_index', 'type', 'week', 'time', 'id', 'word_num', 'push_num', 'web', 'ip', 'title']
w_list.writerow(postInfo)
f_list.close()
for page_index in range(start_index,end_index):
    #### Link web of PTT board and read posts web each page. ####
    if (page_index - start_index) % now_folder_len == (now_folder_len - 1):
        now_folder_cnt = now_folder_cnt + 1
        csv_dir = nowtime_dir+'/'+str(now_time)+'_'+str(now_folder_cnt)+'.csv'
    is_err = True
    err_count = 0
    rs = requests.session()
    while is_err:
        try:
            res = rs.post(over18_web, data=payload18)
            res = rs.get(ptt_web + board_web_tail + str(page_index) +'.html', verify=False)
            #res = requests.get(board_web + str(page_index) +'.html', verify=False)
        except BaseException:
            err_count = err_count + 1
            if err_count > err_max:
                print('ERROR page: ', page_index, 'WEB: ', ptt_web + board_web_tail + str(page_index) +'.html')
                break
            time.sleep(1)
        else:
            is_err = False
    if is_err:
        continue
    
    soup = BeautifulSoup(res.text, "html5lib")
    print("page", page_index, end='\r')
    post_index = 0
    
    with open(csv_dir ,'a', encoding = 'utf-8-sig') as f_list:
        w_list = csv.writer(f_list)
        for entry in soup.select('.r-ent'):
            post_index = post_index + 1
            if post_index < start_post:
                continue
            else:
                start_post = 0
            entry_a = entry.find('a')
            if not entry_a:
                continue
            if not entry_a['href']:
                continue
            
            #### Link web and read content, title and push each post. ####
            is_err = True
            err_count = 0
            po_web = ptt_web + entry_a['href']
            rs = requests.session()
            while is_err:
                try:
                    sub_res = rs.post(over18_web, data=payload18)
                    sub_res = rs.get(po_web, verify=False)
                    #print(po_web)
                    #sub_res = requests.get(po_web)
                except BaseException:
                    err_count = err_count + 1
                    if err_count > err_max:
                        print('ERROR post: ', post_index, 'WEB: ', po_web)
                        break
                    time.sleep(1)
                else:
                    is_err = False
            if is_err:
                continue
                
            # Get all content for this post.
            sub_soup = BeautifulSoup(sub_res.text, "html5lib")
            main_content = sub_soup.select('#main-content')
            if not main_content:
                continue

            po_list = main_content[0].select('.article-meta-value')

            if not po_list:
                continue

            if len(po_list) < 4:
                continue
                
            # Get date and time for this post.
            po_datetime = po_list[3].text
            if not po_datetime:
                continue
            if not len(po_datetime) == 24:
                continue
            po_week = po_datetime[0:3]
            try:
                po_time = time.mktime(time.strptime(po_datetime,"%a %b %d %H:%M:%S %Y"))
            except BaseException:
                continue
                
            # Get content type for this post.
            po_name = po_list[0].text
            if po_name:
                index1 = po_name.find(' (')
                if index1 <= 0:
                    po_id = 'guest'
                else:
                    po_id = po_name[0:index1]
            else:
                po_id = 'guest'
            
            po_title = po_list[2].text
            if po_title:
                index1 = po_title.find('[')
                index2 = po_title.find(']')
                if index1 < 0 or index2 <= index1:
                    po_type = '無題'
                else:
                    po_type = po_title[index1+1:index2]
            else:
                po_type = '無題'
            
            # Get content title for this post.
            po_title = po_title[index2+1:]
            po_pushnum = len(main_content[0].select('.push'))
            
            po_text = ''
            content_cnt = 0
            old_ptt_po = False
            for add_text in main_content[0].strings:
                content_cnt = content_cnt + 1
                # Pass title information.
                if content_cnt <= 8:
                    continue
                # Get the slitted point between content and pushs.
                elif add_text[0:21] == '※ 發信站: 批踢踢實業坊(ptt.cc)':
                    if add_text == '※ 發信站: 批踢踢實業坊(ptt.cc)\n' or add_text == '※ 發信站: 批踢踢實業坊(ptt.cc) \n':
                        old_ptt_po = True
                    else:
                        po_ip = row_1st_keep(add_text)
                        break
                elif old_ptt_po:
                    po_ip = row_1st_keep(add_text)
                    break
                else:
                    po_text = po_text + add_text
            # Read each push.
            po_wordnum = len(po_text)
            po_push = ''
            for add_push in main_content[0].select('.push'):
                for add_pushtext in add_push.strings:
                    po_push = po_push + ',' + add_pushtext

            postInfo = [(page_index - 1) * 20 + post_index, po_type, po_week, po_time, po_id, po_wordnum, po_pushnum, po_web, po_ip, po_title]
            w_list.writerow(postInfo)
            # Save all content for this post.
            with open(content_dir+'/'+str((page_index - 1) * 20 + post_index)+'.csv' ,
                      'a', encoding = 'utf-8-sig') as f_text:
                w_text = csv.writer(f_text)
                w_text.writerow([po_text])
            # Save each push for this post.
            with open(push_dir+'/'+str((page_index - 1) * 20 + post_index)+'.csv' ,
                      'a', encoding = 'utf-8-sig') as f_text:
                w_text = csv.writer(f_text)
                w_text.writerow([po_push])

1520514079.5375044


In [3]:
if start_post < post_index:
    start_post = post_index
print("page",page_index ,"post" ,start_post ,"file", now_time)
f_list.close()

page 2000 post 20 file 1520492468.9048126
